# Importy

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt, timedelta as tmd
from IPython.display import display
from importlib import reload

import APICommunication.config as cfg

import Data.DataLoader
reload(Data.DataLoader)
from Data.DataLoader import *

import Data.DataCleaner
reload(Data.DataCleaner)
from Data.DataCleaner import *

import MarkowitzAnalysis.ReturnAnalysis
reload(MarkowitzAnalysis.ReturnAnalysis)
from MarkowitzAnalysis.ReturnAnalysis import *

import PositionAnalysis.PortfolioPerformance
reload(PositionAnalysis.PortfolioPerformance)
from PositionAnalysis.PortfolioPerformance import *

import PositionAnalysis.PortfolioLoader
reload(PositionAnalysis.PortfolioLoader)
from PositionAnalysis.PortfolioLoader import *

import Plotting.Plotter
reload(Plotting.Plotter)
from Plotting.Plotter import *

import Plotting.EfficientFrontierPlot
reload(Plotting.EfficientFrontierPlot)
from Plotting.EfficientFrontierPlot import *

import Backtest.Backtest
reload(Backtest.Backtest)
from Backtest.Backtest import *

from warnings import filterwarnings
filterwarnings('ignore')

# Działanie właściwego programu

### Scenariusz 0. Aktualizacja danych do obecnej daty

In [ ]:
start = '2025-01-25'

print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

dataloader = DataLoader(cfg.user_id, cfg.pwd)
data = dataloader.loadInstrumentsData(start)

### Scenariusz 1. Analiza obecnej pozycji przez łączenie z API

In [ ]:
print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

pl = PortfolioLoader('Pozycja od 10.12.2024 do 10.03.2025')

dataloader = DataLoader(cfg.user_id, cfg.pwd)
info = dataloader.loadInstrumentsInfo()

currentTrades = dataloader.getCurrentTrades(info, weights = pl.portfolio)
display(currentTrades.getSummary())
print(f"Zwrot z portfela: {currentTrades.getPCTReturn()}%")
print(f"Zysk z inwestycji: {currentTrades.getPLNReturn()} PLN")

### Scenariusz 2. Wybór nowej pozycji

In [ ]:
start, end = '2024-01-01', '2025-01-09'

print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

dataloader = DataLoader(cfg.user_id, cfg.pwd)
data = dataloader.loadInstrumentsData(start, end)
info = dataloader.loadInstrumentsInfo()

datacleaner = DataCleaner(data, info)

freq = '8M'
returnRates = datacleaner.getReturnRates(freq)
mo = MarkowitzOptimization(returnRates, freq)
mo.getOptimalWeights(model='max_sharpe', risk_method='oas')
portfolio = mo.getPortfolio()
portfolio.getSummary()

In [ ]:
portfolio.calculateRealPortfolio(1200)

### Scenariusz 3. Analiza konkretnego portfela

#### Wariant 1. Podajemy dane ręcznie

In [ ]:
print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

portfolio = {'P500.DE': 32.56, 
             '4GLD.DE': 17.47, 
             'FLXI.DE': 45.06, 
             'ESD.FR': 4.91}
freq = '3M'

start, end = '2023-07-01', '2024-12-10'

dataloader = DataLoader(cfg.user_id, cfg.pwd)
data = dataloader.loadInstrumentsData(start, end)
info = dataloader.loadInstrumentsInfo()

datacleaner = DataCleaner(data, info, load_only=list(portfolio.keys()))
data = datacleaner.getBidPrice()
returnRates = datacleaner.getReturnRates(freq)
pp = PortfolioPerformance(portfolio, returnRates, freq, 'empirical', 'max_utility',data=data)
pp.getSummary()

In [14]:
SaveDict(pp.getStatDict(), 'Pozycja od 10.12.2024 do 10.03.2025', 'Positions')

#### Wariant 2. Wygrywamy automatycznie zapisany portfel

In [ ]:
print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

start, end = '2024-01-01', '2025-01-13'
pl = PortfolioLoader('Pozycja od 10.12.2024 do 10.03.2025')
pl.getSummary()

dataloader = DataLoader(cfg.user_id, cfg.pwd)
data = dataloader.loadInstrumentsData(start, end)
info = dataloader.loadInstrumentsInfo()

datacleaner = DataCleaner(data, info, load_only=pl.symbols)
data = datacleaner.getBidPrice()
returnRates = datacleaner.getReturnRates(pl.freq)

# pp = pl.getPortfolio(returnRates, data)

### Scenariusz 4. Wykresy dla konkretnego portfela

In [ ]:
print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

pl = PortfolioLoader('Pozycja od 10.12.2024 do 10.03.2025')
freq = '6M'
print()
pl.getSummary()
print()
start, end = '2023-01-01', '2025-01-13'
dataloader = DataLoader(cfg.user_id, cfg.pwd)
data = dataloader.loadInstrumentsData(start, end)
info = dataloader.loadInstrumentsInfo()

datacleaner = DataCleaner(data, info, load_only=pl.symbols)
data = datacleaner.getBidPrice()
returnRates = datacleaner.getReturnRates(freq)

plotter = Plotter(pl.portfolio,
                  data,
                  returnRates,
                  freq,
                  pl.model,
                  pl.risk_method,
                  compare='ema_2w')
plotter.plot(show_instruments=True)

### Scenariusz 5. Wykres "Efficient Frontier"

In [ ]:
start, end = '2020-01-01', '2025-01-13'

print(f"[INFO] Rozpoczynam pracę programu: {now(False)}")

dataloader = DataLoader(cfg.user_id, cfg.pwd)
data = dataloader.loadInstrumentsData(start, end)
info = dataloader.loadInstrumentsInfo()

datacleaner = DataCleaner(data, info)

In [ ]:
freq = '1Y'
returnRates = datacleaner.getReturnRates(freq)
ef = EfficientFrontierPlot(returnRates, freq, Npoints=80)

In [ ]:
ef.plot(Nrandom=5000)

### Scenariusz 6: Backtest modelu

In [ ]:
# Input:
start, end = '2023-07-01', '2025-02-03'
freq = '3M'


bt = Backtest(start, end, freq, len_train=3)
backtest_summary = bt.getSummary()
backtest_summary

# Brudnopis

### Backtest - pierwsze kroki

In [ ]:
# Input:
start, end = '2020-01-01', '2025-01-30'
freq = '3M'

dates = [x.strftime("%Y-%m-%d") for x in pd.date_range(start, end, freq=freq)]
len_train = 4 # to powinno być jakąś funkcją 'freq'
len_train += 1 # stopy zwrotu nie obliczają się dla pierwszego okresu treningowego

######################################################################
train = []
test = []
for i, x in enumerate(dates):
    
    if i+len_train+1 >= len(dates):
        break
    
    start_train = dates[i]
    end_train = dates[i+len_train]
    end_test = dates[i+len_train+1]
    
    train.append((start_train, end_train))
    test.append((end_train, end_test))

assert len(train) == len(test)

######################################################################
summary_dict = {}

for i in range(len(train)):
    
    # Okres treningowy
    start_train, end_train = train[i]
    
    # Okres testowy
    start_test, end_test = test[i]
    
    # Ładujemy całe dane na raz
    dataloader = DataLoader(cfg.user_id, cfg.pwd)
    
    print(f"[INFO] Okres {i+1} z {len(train)}. Pobieramy dane od {start_train} do {end_test}: {now(False)}")
    data = dataloader.loadInstrumentsData(start_train, end_test)
    info = dataloader.loadInstrumentsInfo()
    
    datacleaner = DataCleaner(data, info, verbose=False)
    returnRates = datacleaner.getReturnRates(freq)
    
    # Trening
    print(f"\t[INFO] Trening w okresie od {start_train} do {end_train}")

    train_indices = returnRates.index.isin(pd.date_range(start_train, end_train))
    mo = MarkowitzOptimization(returnRates.loc[train_indices, :], freq, verbose=False)
    mo.getOptimalWeights(model='max_sharpe', risk_method='oas')
    pp = mo.getPortfolio()
    expected_return = pp.portfoliomean
    conf_int = getQuantiles(pp.portfolioReturn, q=0.05)
    sharpe_ratio = pp.sharpe_ratio
    
    print(f"\tOczekiwany zwrot treningowy: {expected_return:.2%}")
    print(f"\tSharpe Ratio: {sharpe_ratio:.2f}")
    print()
    portfolio = pp.portfolio
    
    # Test
    print(f"\t[INFO] Test w okresie od {start_test} do {end_test}")

    test_indices = returnRates.index.isin(pd.date_range(start_train, end_train))
    pp = PortfolioPerformance(portfolio, returnRates.loc[test_indices, :], freq, 'empirical', 'max_utility')
    true_return = pp.portfolioReturn.iloc[-1]
    
    # if sharpe_ratio >= 1.0:
    #     if true_return < conf_int[0]: summary_dict['below'] += 1
    #     elif (true_return >= conf_int[0]) and (true_return < expected_return): summary_dict['lower_half'] += 1
    #     elif (true_return >= expected_return) and (true_return < conf_int[1]): summary_dict['upper_half'] += 1
    #     elif true_return > conf_int[1]: summary_dict['above'] += 1
    #     else: print('Jakiś idiotyczny błąd xd')
    
    print(f"\tRzeczywisty zwrot w okresie testowym: {true_return:.2%}")
    print(f"\tTreningowy przedział ufności: [{conf_int[0]:.2%}, {conf_int[1]:.2%}]")
    print()
    
    summary_dict[i+1] = {'expected_return': expected_return,
                       'true_return': true_return,
                       'conf_int_low': conf_int[0],
                       'conf_int_high': conf_int[1],
                       'sharpe_ratio': sharpe_ratio}

summary = pd.DataFrame(summary_dict).T
display(summary)

In [ ]:
summary['in_conf_int'] = summary['true_return'].between(summary['conf_int_low'], summary['conf_int_high'])
summary.insert(loc=2, column='error', value=(summary['true_return'] - summary['expected_return']))
summary

### Backtest - drugie kroki

In [3]:
# Input:
start, end = '2023-07-01', '2025-02-03'
freq = '3M'


bt = Backtest(start, end, freq, len_train=3)
backtest_summary = bt.getSummary()
backtest_summary

[INFO] Okres 1 z 1. Pobieramy dane od 2023-07-31 do 2024-10-31: 2025-02-04 07:55:36
	[INFO] Trening w okresie od 2023-07-31 do 2024-07-31
	Oczekiwany zwrot treningowy: 7.09%
	Sharpe Ratio: 1.39

	[INFO] Test w okresie od 2024-07-31 do 2024-10-31
	Rzeczywisty zwrot w okresie testowym: 4.38%
	Treningowy przedział ufności: [0.50%, 12.98%]



,ExpectedReturn,TrueReturn,Error,ConfIntLow,ConfIntHigh,SharpeRatio,InConfInt
1,0.070934,0.043789,-0.027145,0.004952,0.129751,1.392278,True
